In [4]:
import geopandas as gpd
from pyproj import CRS
import matplotlib.pyplot as plt

# Filepaths
grid_fp = r'data/
roads_fp =r'data/
metro_fp =r'data/

# Read files
grid = gpd.read_file(grid_fp)
roads = gpd.read_file(roads_fp)
metro = gpd.read_file(metro_fp)

DriverError: TravelTimes_to_5975375_RailwayStation.shp: No such file or directory